In [1]:
import fsspec
import ujson

fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True)
fs_write = fsspec.filesystem('')

In [2]:
hrr_subhourly_member_files = fs_read.glob('s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf*.grib2')

files = sorted(['s3://'+f for f in hrr_subhourly_member_files])
print(f'Read {len(files)} HRRR files')

Read 19 HRRR files


In [17]:
import ujson
from kerchunk.grib2 import scan_grib
from kerchunk.combine import MultiZarrToZarr

afilter = {"typeOfLevel": "surface", "stepType": "instant"}
so = {"anon": True}
json_dir = "idx/"


def gen_chunks(file_url):
    messages = scan_grib(
        file_url, storage_options=so, filter=afilter
    )  # create the reference using scan_grib
    mzz = MultiZarrToZarr(
        messages,
        concat_dims=["valid_time"],
        identical_dims=["latitude", "longitude", "x", "y"],
        remote_protocol='s3',
        remote_options=so,
    )
    return mzz

In [18]:
mzz = gen_chunks(files[1])
mzz

In [19]:
d = mzz.translate()

In [20]:
import xarray as xr

fs = fsspec.filesystem("reference", fo=d, remote_protocol='s3', remote_options={'anon':True})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={'valid_time':1})
ds

<xarray.Dataset>
Dimensions:     (valid_time: 4, y: 1059, x: 1799)
Coordinates:
    latitude    (y, x) float64 dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude   (y, x) float64 dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    step        (valid_time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    surface     (valid_time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    time        (valid_time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * valid_time  (valid_time) datetime64[ns] 2023-09-27T00:15:00 ... 2023-09-2...
Dimensions without coordinates: y, x
Data variables: (12/16)
    cfrzr       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    cicep       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    cpofp       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    crain       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    csnow       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    dlwrf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    ...          ...
    sp          (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    ulwrf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    uswrf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vbdsf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vddsf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vis         (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
Attributes:
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_edition:            2
    GRIB_subCentre:          0
    institution:             US National Weather Service - NCEP

In [21]:
next_timestep_idx_files = hrr_subhourly_member_files = fs_read.glob('s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t01z.wrfsubhf*.grib2.idx')
idx_files = sorted(['s3://'+f for f in next_timestep_idx_files])
print(f'Read {len(idx_files)} HRRR IDX files')

Read 19 HRRR IDX files


In [24]:
refs = d['refs']

In [25]:
def parse_idx(file_url):
    messages = []
    with fs_read.open(file_url, 'r') as f:
        for line in f.readlines():
            parts = str(line).split(':')
            messages.append({
                'index': int(parts[0]),
                'offset': int(parts[1]),
                'date': parts[2].split('=')[1],
                'var': parts[3].lower(),
                'surface': parts[4],
                'valid_time': parts[5],
            })
    return messages

In [26]:
idx_refs = parse_idx(idx_files[1])

We are only dealing with surface variables in this example, so lets assume one variable for each key. More sophisticated usage in the future will have to deal with the fact that there are multiple variables for each key.  

In [33]:
ref_vars = set([k.split('/')[0] for k in refs.keys()])
ref_vars.remove('.zattrs')
ref_vars.remove('.zgroup')
ref_vars

{'cfrzr',
 'cicep',
 'cpofp',
 'crain',
 'csnow',
 'dlwrf',
 'dswrf',
 'gust',
 'latitude',
 'longitude',
 'orog',
 'prate',
 'sp',
 'step',
 'surface',
 'time',
 'ulwrf',
 'uswrf',
 'valid_time',
 'vbdsf',
 'vddsf',
 'vis'}

In [34]:
matching_vars = [r for r in idx_refs if (r['var'] in ref_vars and r['surface'] == 'surface')]
f'Found {len(matching_vars)} matching variables'

'Found 64 matching variables'

Okay now that we have the matching variables, we need to get the time array for the existing references. Im not sure how to decode these yet. 

Once they are decoded, we need to match up the variables from the idx files then append the new chunk to the exisiting array using the offsets found from the idx files.

In [35]:
import base64

base64.b64decode(refs['time/0'].encode('utf-8')[7:])

b'\x80p\x13e\x00\x00\x00\x00'